In [4]:
import pickle


with open('proc_b_trg_emb.p', 'rb') as fp:
    trg_emb = pickle.load(fp)
with open('proc_b_trg_word.p', 'rb') as fp:
    trg_word = pickle.load(fp)
    
    
with open('proc_b_src_emb.p', 'rb') as fp:
    src_emb = pickle.load(fp)
with open('proc_b_src_word.p', 'rb') as fp:
    src_word = pickle.load(fp)



In [5]:
import numpy as np

#CUPY is for GPU usage
#import cupy

def get_array_module(x):
    #if cupy is not None:
    #    return cupy.get_array_module(x)
    #else:
        return np
    
def normalize_matrix(matrix):
    xp = get_array_module(matrix)
    norms = xp.sqrt(xp.sum(xp.square(matrix), axis=1))
    norms[norms == 0] = 1
    norms = norms.reshape(-1, 1)
    matrix /= norms[:]
    return matrix


def calculate_nn(given_source_word, src_word, norm_src_emb, trg_word, norm_trg_emb, n_neighbors=5):
    """
    Need normed embeddings!
    """
    given_source_index = src_word[given_source_word]

    # Calculate Cos Similarity
    norm_src_word_emb = norm_src_emb[[given_source_index]]
    similarity_cos = np.dot(norm_src_word_emb, np.transpose(norm_trg_emb))
    
    # Find Closest Neighbors
    most_similar_trg_index = np.argsort(-similarity_cos[[0]])[:n_neighbors].tolist()[0][:n_neighbors]
    
    return most_similar_trg_index

In [6]:
#inverse_src_word = {index: word for word, index in src_word.items()}
inverse_trg_word = {index: word for word, index in trg_word.items()}

norm_trg_emb = normalize_matrix(trg_emb)
norm_src_emb = normalize_matrix(src_emb)

nn = calculate_nn("cat", src_word, norm_src_emb, trg_word, norm_trg_emb, n_neighbors=5)

for single_neighbor in nn:
    print(inverse_trg_word[single_neighbor])

katze
hund
katzen
hauskatze
kater
